## Imports

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyspark
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import DataFrame,SparkSession

from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, roc_curve, auc
import seaborn as sns
from sklearn.metrics import roc_curve, auc, confusion_matrix, classification_report, precision_recall_curve
from scipy.stats import ks_2samp
from scipy import stats

import lightgbm as lgbm
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import shap

## Funções Objetivo

In [0]:
def objective_lgbm(params):
    model = lgbm.LGBMClassifier(
        num_leaves=int(params['num_leaves']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        min_child_samples=int(params['min_child_samples']),
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]),
        objective="binary",
        random_state=42
    )
    
    model.fit(X_train, y_train)

    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    auc_score = roc_auc_score(y_test, y_pred_proba)
    
    return {'loss': -auc_score, 'status': STATUS_OK}

In [0]:
def plot_roc_curve(model, X_test, y_test):
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

def plot_confusion_matrix(model, X_test, y_test, threshold):
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = [1 if pred >= threshold else 0 for pred in y_pred_proba]
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

def plot_feature_importances(model, feature_names):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    plt.figure(figsize=(12, 6))
    plt.title('Feature Importances')
    plt.bar(range(len(importances)), importances[indices], align='center')
    plt.xticks(range(len(importances)), [feature_names[i] for i in indices], rotation=90)
    plt.tight_layout()
    plt.show()

def plot_classification_report(model, X_test, y_test, threshold):
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    y_pred = [1 if pred >= threshold else 0 for pred in y_pred_proba]
    report = classification_report(y_test, y_pred, output_dict=True)
    
    plt.figure(figsize=(10, 6))
    sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True, cmap='Blues')
    plt.title('Classification Report')
    plt.show()

In [0]:
def ks_grouped_by_safra(df, target_col, prob_col, safras):
    ks_df = pd.DataFrame(columns=['safra', 'ks_stat', 'p_value'])
    for safra in sorted(safras):
        df_safra = df[df['safra'] == safra]
        ks_stat, p_value = stats.ks_2samp(df_safra[df_safra[target_col] == 1][prob_col], df_safra[df_safra[target_col] == 0][prob_col])
        ks_df = ks_df.append({'safra': safra, 'ks_stat': ks_stat, 'p_value': p_value}, ignore_index=True)
    return ks_df

In [0]:
def calculate_churn_percentage(df, target_col, pred_col, pred_col_bin, threshold):
    actual_churn_rate = df[target_col].mean()
    df[pred_col_bin] = (df[pred_col] >= threshold).astype(int)
    predicted_churn_rate = df[pred_col_bin].mean()
    return actual_churn_rate, predicted_churn_rate

def calculate_churn_by_safra(df, target_col, pred_col):
    grouped = df.groupby('safra').agg(
        actual_churn_rate=(target_col, 'mean'),
        predicted_churn_rate=(pred_col, 'mean')
    ).reset_index()
    return grouped

## Base de Modelagem

In [0]:
base_spine = spark.table("sand_riscos_pm_pf.T789778_base_final_dm_v3").filter(F.col('is_auto_renew_1m').isin(0)).sample(0.25).drop(*['features', 'is_auto_renew_1m_index'])
base_spine

## Variáveis

In [0]:
id_vars = ['msno', 'safra']
target = ['target']

In [0]:
variaveis_rf = ['payment_plan_days_1m',
 'actual_amount_paid_1m',
 'actual_amount_paid_max_2m',
 'plan_list_price_1m',
 'plan_list_price_avg_3m',
 'actual_amount_paid_avg_2m',
 'city_1m_index',
 'plan_list_price_avg_2m',
 'num_unq_max_4m',
 'actual_amount_paid_median_4m',
 'num_unq_max_3m',
 'num_unq_avg_3m',
 'plan_list_price_min_2m',
 'plan_list_price_median_2m',
 'actual_amount_paid_avg_3m',
 'total_secs_min_2m',
 'completion_rate_100_max_4m',
 'plan_list_price_max_2m',
 'num_100_median_2m',
 'num_unq_max_2m',
 'num_25_max_4m',
 'num_unq_avg_2m',
 'actual_amount_paid_min_4m',
 'completion_rate_100_median_2m',
 'num_unq_avg_4m',
 'total_secs_median_2m',
 'completion_rate_100_min_2m',
 'num_50_max_4m',
 'num_unq_median_3m',
 'num_985_max_4m',
 'total_secs_avg_2m',
 'account_time_1m',
 'completion_rate_100_min_3m',
 'num_unq_min_3m',
 'completion_rate_25_max_4m',
 'total_secs_min_3m',
 'payment_method_id_1m_index',
]

In [0]:
len(variaveis_rf)

## Treino - Teste - Validação

In [0]:
train_data = base_spine.filter(F.col('safra').between(201505,201604)).select(id_vars+variaveis_rf+target).toPandas()
test_data = base_spine.filter(F.col('safra').between(201605,201607)).select(id_vars+variaveis_rf+target).toPandas()
validation_data = base_spine.filter(F.col('safra').between(201608,201611)).select(id_vars+variaveis_rf+target).toPandas()

In [0]:
X_train, y_train = train_data[variaveis_rf], train_data[target]
X_test, y_test = test_data[variaveis_rf], test_data[target]
X_val, y_val = validation_data[variaveis_rf], validation_data[target]

In [0]:
scale_pos_weight = (y_train.value_counts()[0] / y_train.value_counts()[1])
scale_pos_weight

# LIGHTGBM

## Optimização de Hiperparâmetros

- 'num_leaves':  Número máximo de folhas
- 'max_depth': Profundidade máxima das árvores
- 'learning_rate': Taxa de aprendizado (apliquei uma log-uniforme)
- 'subsample': Fração de amostras usadas para treino
- 'colsample_bytree': Fração de features usadas por árvore
- 'min_child_samples':  Número mínimo de amostras por folha

In [0]:
space_lgbm = {
    'num_leaves': hp.choice('num_leaves', range(20, 150)),
    'max_depth': hp.choice('max_depth', range(3, 10)),
    'learning_rate': hp.loguniform('learning_rate', -5, 0),
    'subsample': hp.uniform('subsample', 0.6, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
    'min_child_samples': hp.choice('min_child_samples', range(10, 30)),
    'reg_alpha': hp.loguniform('reg_alpha', -4, 0),
    'reg_lambda': hp.loguniform('reg_lambda', -4, 0)
}

In [0]:
trials_lgbm = Trials()
best_lgbm = fmin(fn=objective_lgbm,
                space=space_lgbm,
                algo=tpe.suggest,
                max_evals=50,  # Número máximo de avaliações
                trials=trials_lgbm)

In [0]:
# best_lgbm = {'colsample_bytree': 0.7552694502661478, 'learning_rate': 0.017441402328506637, 'max_depth': 4, 'min_child_samples': 36, 'num_leaves': 33, 'reg_alpha': 0.018266132285821874, 'reg_lambda': 0.014404655497649174, 'subsample': 0.7893496096936965}

In [0]:
print("Melhores hiperparâmetros para LightGBM:")
print(best_lgbm)

In [0]:
final_lgbm_model = lgbm.LGBMClassifier(
    num_leaves=int(best_lgbm['num_leaves']),
    max_depth=int(best_lgbm['max_depth']),
    learning_rate=best_lgbm['learning_rate'],
    subsample=best_lgbm['subsample'],
    colsample_bytree=best_lgbm['colsample_bytree'],
    min_child_samples=int(best_lgbm['min_child_samples']),
    reg_alpha=best_lgbm['reg_alpha'],
    reg_lambda=best_lgbm['reg_lambda'],
    scale_pos_weight=scale_pos_weight,
    objective="binary",
    random_state=42
).fit(X_train, y_train)

In [0]:
final_lgbm_model

In [0]:
y_pred_lgbm = final_lgbm_model.predict_proba(X_test)[:, 1]
auc_lgbm = roc_auc_score(y_test, y_pred_lgbm)
print(f"AUC LightGBM (melhores hiperparâmetros): {auc_lgbm}")

## Escorar Bases

In [0]:
train_data_escorado = final_lgbm_model.predict_proba(train_data[variaveis_rf])[:,1]
test_data_escorado = final_lgbm_model.predict_proba(test_data[variaveis_rf])[:,1]
val_data_escorado = final_lgbm_model.predict_proba(validation_data[variaveis_rf])[:,1]

In [0]:
train_data['score_lgbm'] = train_data_escorado
test_data['score_lgbm'] = test_data_escorado
validation_data['score_lgbm'] = val_data_escorado

## Análises

In [0]:
plt.hist(train_data['score_lgbm'], bins=100, alpha=0.5, label='Train')
plt.hist(test_data['score_lgbm'], bins=100, alpha=0.5, label='Test')
plt.hist(validation_data['score_lgbm'], bins=100, alpha=0.5, label='Validation')
plt.title('Score LightGBM')
plt.xlabel('Score')
plt.ylabel('Frequência')
plt.legend()

In [0]:
plt.hist(train_data[train_data['target']==0]['score_lgbm'], bins=100, alpha=0.5, label='Train')
plt.hist(test_data[test_data['target']==0]['score_lgbm'], bins=100, alpha=0.5, label='Test')
plt.hist(validation_data[validation_data['target']==0]['score_lgbm'], bins=100, alpha=0.5, label='Validation')
plt.title('Score LightGBM Target 0')
plt.xlabel('Score')
plt.ylabel('Frequência')
plt.legend()

In [0]:
plt.hist(train_data[train_data['target']==1]['score_lgbm'], bins=100, alpha=0.5, label='Train')
plt.hist(test_data[test_data['target']==1]['score_lgbm'], bins=100, alpha=0.5, label='Test')
plt.hist(validation_data[validation_data['target']==1]['score_lgbm'], bins=100, alpha=0.5, label='Validation')
plt.title('Score LightGBM Target 1')
plt.xlabel('Score')
plt.ylabel('Frequência')
plt.legend()

In [0]:
thresholds = np.arange(0.0, 1.05, 0.005)
f1_scores = []
best_threshold = 0
best_f1_score = 0

for threshold in thresholds:
    y_pred_bin = np.where(test_data['score_lgbm'] >= threshold, 1, 0)

    f1 = f1_score(test_data['target'], y_pred_bin)
    f1_scores.append(f1)

    if f1 > best_f1_score:
        best_f1_score = f1
        best_threshold = threshold


print(f"Melhor Threshold: {best_threshold:.2f}")
print(f"Melhor F1-Score: {best_f1_score:.4f}")

In [0]:
thresholds_f1_scores = list(zip(thresholds, f1_scores))
thresholds_f1_scores

In [0]:
plt.figure(figsize=(10, 6))
plt.scatter(thresholds, f1_scores)
plt.xlabel('Thresholds')
plt.ylabel('F1-Scores')

max_f1_score = max(f1_scores)
max_f1_index = f1_scores.index(max_f1_score)
max_f1_threshold = thresholds[max_f1_index]

plt.annotate(f'Max F1-Score: {max_f1_score:.4f}\n(x, y) = ({max_f1_threshold:.2f}, {max_f1_score:.4f})', 
             xy=(max_f1_threshold, max_f1_score), 
             xytext=(max_f1_threshold, max_f1_score + 0.09),
             arrowprops=dict(facecolor='black', shrink=0.05))

plt.show()

In [0]:
y_pred_proba = test_data['score_lgbm']
precision, recall, _ = precision_recall_curve(test_data['target'], y_pred_proba)

plt.figure(figsize=(10, 6))
plt.plot(recall, precision, marker='o')
plt.title('Recall vs Precision')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [0]:
def calculate_model_statistics(y_true, y_pred_proba, threshold):
    stats = {}

    stats['AUC'] = roc_auc_score(y_true, y_pred_proba)
    
    y_pred_binary = [1 if pred >= threshold else 0 for pred in y_pred_proba]
    
    stats['Precision'] = precision_score(y_true, y_pred_binary)
    
    stats['Recall'] = recall_score(y_true, y_pred_binary)

    stats['F1-Score'] = f1_score(y_true, y_pred_binary)
    
    stats['KS'] = ks_2samp(y_pred_proba[y_true == 1], y_pred_proba[y_true == 0]).statistic
    
    return stats

In [0]:
calculate_model_statistics(train_data['target'], train_data['score_lgbm'], best_threshold)

In [0]:
calculate_model_statistics(test_data['target'], test_data['score_lgbm'], best_threshold)

In [0]:
calculate_model_statistics(validation_data['target'], validation_data['score_lgbm'], best_threshold)

In [0]:
train_stats = calculate_model_statistics(train_data['target'], train_data['score_lgbm'], best_threshold)
test_stats = calculate_model_statistics(test_data['target'], test_data['score_lgbm'], best_threshold)
val_stats = calculate_model_statistics(validation_data['target'], validation_data['score_lgbm'], best_threshold)

train_stats_df = pd.DataFrame([train_stats])
test_stats_df = pd.DataFrame([test_stats])
val_stats_df = pd.DataFrame([val_stats])

train_stats_df['Period'] = 'Train'
test_stats_df['Period'] = 'Test'
val_stats_df['Period'] = 'Validation'

combined_stats = pd.concat([train_stats_df, test_stats_df, val_stats_df], ignore_index=True)
combined_stats = combined_stats[['Period'] + [col for col in combined_stats.columns if col != 'Period']]
combined_stats.iloc[:, 1:] = combined_stats.iloc[:, 1:].applymap(lambda x: f"{x:.2%}")
display(combined_stats)

In [0]:
plot_roc_curve(final_lgbm_model, X_test, y_test)
plot_confusion_matrix(final_lgbm_model, X_test, y_test, threshold=best_threshold)
plot_feature_importances(final_lgbm_model, X_test.columns)
plot_classification_report(final_lgbm_model, X_test, y_test, threshold=best_threshold)

In [0]:
plot_roc_curve(final_lgbm_model, X_val, y_val)
plot_confusion_matrix(final_lgbm_model, X_val, y_val, threshold=best_threshold)
plot_feature_importances(final_lgbm_model, X_val.columns)
plot_classification_report(final_lgbm_model, X_val, y_val, threshold=best_threshold)

In [0]:
feature_importances = final_lgbm_model.feature_importances_
features = X_train.columns

importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})

importance_df = importance_df.sort_values(by='Importance', ascending=False)

plt.figure(figsize=(15, 12))
plt.barh(importance_df['Feature'], importance_df['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance - LightGBM')
plt.gca().invert_yaxis()
plt.show()

In [0]:
%python
import shap

# Initialize the SHAP explainer
explainer = shap.Explainer(final_lgbm_model)

# Calculate SHAP values for the training data
shap_values = explainer(train_data[variaveis_rf])

# Convert the training data to a numpy array
train_data_np = train_data[variaveis_rf].to_numpy()

# Plot the SHAP summary plot
# shap.summary_plot(shap_values.values, train_data_np, feature_names=variaveis_rf)

In [0]:
train_data_np.shape

In [0]:
shap_values.values[:, :, 0]

In [0]:
shap.summary_plot(shap_values.values[:, :, 0], train_data_np, feature_names=variaveis_rf)

In [0]:
importance_df['Importance'] = (importance_df['Importance'] / importance_df['Importance'].sum()) * 100
display(importance_df)

In [0]:
s1 = train_data['safra'].unique()
s2 = test_data['safra'].unique()
s3 = validation_data['safra'].unique()

ks_s1 = ks_grouped_by_safra(train_data, 'target', 'score_lgbm', sorted(s1))
ks_s2 = ks_grouped_by_safra(test_data, 'target', 'score_lgbm', sorted(s2))
ks_s3 = ks_grouped_by_safra(validation_data, 'target', 'score_lgbm', sorted(s3))

ks_union = pd.concat([ks_s1, ks_s2, ks_s3])

# Plotting the KS statistics
plt.figure(figsize=(10, 6))
plt.plot(ks_union['safra'].astype(str), ks_union['ks_stat']*100, marker='o')
# plt.axvline(x=str(sorted(s2)[0]), color='r', linestyle='--')
# plt.axvline(x=str(sorted(s3)[0]), color='g', linestyle='--')
plt.ylim(0, 50)
plt.xlabel('Safra')
plt.ylabel('KS Statistic')
plt.title('KS Statistic by Safra')
plt.xticks(rotation=45)
plt.show()

In [0]:
train_data['score_lgbm'] = train_data_escorado
test_data['score_lgbm'] = test_data_escorado
validation_data['score_lgbm'] = val_data_escorado

train_data['y_pred_lgbm'] = (train_data['score_lgbm'] > best_threshold).astype(int)
test_data['y_pred_lgbm'] = (test_data['score_lgbm'] > best_threshold).astype(int)
validation_data['y_pred_lgbm'] = (validation_data['score_lgbm'] > best_threshold).astype(int)

In [0]:
train_churn_actual, train_churn_predicted = calculate_churn_percentage(train_data, 'target', 'score_lgbm', 'y_pred', best_threshold)
test_churn_actual, test_churn_predicted = calculate_churn_percentage(test_data, 'target', 'score_lgbm', 'y_pred', best_threshold)
val_churn_actual, val_churn_predicted = calculate_churn_percentage(validation_data, 'target', 'score_lgbm', 'y_pred', best_threshold)

churn_stats = pd.DataFrame({
    'Period': ['Train', 'Test', 'Validation'],
    'Actual Churn Rate': [train_churn_actual, test_churn_actual, val_churn_actual],
    'Predicted Churn Rate': [train_churn_predicted, test_churn_predicted, val_churn_predicted]
})

churn_stats.iloc[:, 1:] = churn_stats.iloc[:, 1:].applymap(lambda x: f"{x:.2%}")
display(churn_stats)

train_churn_by_safra = calculate_churn_by_safra(train_data, 'target', 'y_pred')
test_churn_by_safra = calculate_churn_by_safra(test_data, 'target', 'y_pred')
val_churn_by_safra = calculate_churn_by_safra(validation_data, 'target', 'y_pred')

churn_by_safra = pd.concat([
    train_churn_by_safra.assign(Period='Train'),
    test_churn_by_safra.assign(Period='Test'),
    val_churn_by_safra.assign(Period='Validation')
], ignore_index=True)

churn_by_safra['actual_churn_rate'] *= 100
churn_by_safra['predicted_churn_rate'] *= 100

plt.figure(figsize=(12, 6))
sns.barplot(data=churn_by_safra.melt(id_vars=['safra', 'Period'], value_vars=['actual_churn_rate', 'predicted_churn_rate']),
            x='safra', y='value', hue='variable', ci=None)
plt.title('Churn Rates by Safra')
plt.xlabel('Safra')
plt.ylabel('Churn Rate (%)')
plt.legend(title='Churn Rate Type', labels=['Actual Churn Rate', 'Predicted Churn Rate'])
plt.xticks(rotation=45)
plt.show()